# Download and import librairies

In [1]:
# Import Librairies
# !pip install gspread_dataframe gspread oauth2client
# !pip install gspread-pandas
# !{sys.executable} -m pip install pygsheets
from gspread_pandas import Spread, Client
from google.oauth2.service_account import Credentials
import pandas as pd
import re
import sys
import json
import gspread
import pygsheets
import csv
from gspread_dataframe import get_as_dataframe, set_with_dataframe
from oauth2client.service_account import ServiceAccountCredentials
from collections import defaultdict
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction import text
from sklearn.cluster import DBSCAN

# Import our Google Search Console data from a CSV file

In [2]:
# Import our Google Search Console CSV Data into a Dataframe
df = pd.read_csv('Queries.csv', delimiter=',')
df.head(20)


,Top queries,Clicks,Impressions,CTR,Position
0,adelphi living,255,765,33.33%,1.06
1,adelphi,15,314,4.78%,10.04
2,sil providers perth,11,98,11.22%,2.20
3,supported independent living perth,8,327,2.45%,8.10
4,disability accommodation perth,4,645,0.62%,21.36
5,sda providers perth,4,39,10.26%,7.21
6,ndis providers near me,3,54,5.56%,1.00
7,adelphi living mandurah,3,18,16.67%,1.06
8,sil ndis,2,906,0.22%,37.77
9,ndis housing perth,2,454,0.44%,10.86


## Create a list of keywords from our GSC data

In [3]:
# Define a variable to hold our 'Top queries' column only
top_queries = df['Top queries']

# Create a list of keywords and populate it from our dataframe
keywords = []

for x in top_queries:
  keywords.append(x)

# Display keywords list
keywords

['adelphi living',
 'adelphi',
 'sil providers perth',
 'supported independent living perth',
 'disability accommodation perth',
 'sda providers perth',
 'ndis providers near me',
 'adelphi living mandurah',
 'sil ndis',
 'ndis housing perth',
 'ndis providers perth',
 'disability supported accommodation perth',
 'sil accommodation perth',
 'ndis',
 'disability housing perth',
 'specialist disability accommodation perth',
 'sil',
 'ndis events perth',
 'ndis short term accommodation',
 'disability service providers perth',
 'disability accommodation wa',
 'medium term accommodation ndis',
 'medium term accommodation',
 'disability housing',
 'ndis respite accommodation',
 'medium term accommodation perth',
 'ndis sda housing',
 'disability homes',
 'sil housing',
 'adelphi apartments perth',
 'adelphi living jobs',
 'ndis robust housing',
 'ndis wa providers',
 'sda living',
 'disability services near me',
 'adelphi care',
 'support coordinators',
 'sda payment',
 'assisted living for 

# Start clustering our keywords using Machine Learning sklearn library

In [4]:
# Define our keyword clustering model with sklearn and TfidfVectorizer (using DBSCAN and not Kmean)
tfidf_vectorizer = TfidfVectorizer(max_df=0.2, max_features=10000,min_df=0.01,use_idf=True, ngram_range=(1,2))

# Import our keyword list for Keyword Clustering
tfidf_matrix = tfidf_vectorizer.fit_transform(keywords)

# Define settings for cluster adjustment (number and size of clusters)
ds = DBSCAN(eps=0.7, min_samples=5).fit(tfidf_matrix)
clusters = ds.labels_

## Create a dataframe to hold our keyword clusters

In [5]:
# Create a dataframe for our clusters
cluster_df = pd.DataFrame(clusters, columns=['Cluster Number'])

# Merge our cluster df with our top queries df
result = pd.merge(cluster_df, top_queries, left_index=True, right_index=True)

# We group our keywords by cluster number and separate them with a comma
clusters_queries = result.groupby(['Cluster Number'])['Top queries'].apply(', '.join).reset_index()

# Extend Column and Row width for a better readability 
pd.set_option('max_colwidth', 400)
pd.set_option('max_rows', 100)

clusters_queries


,Cluster Number,Top queries
0,-1,"adelphi living mandurah, sil housing, ndis wa providers, adelphi care, assisted living for schizophrenia near me, adelphi housing, ndis support provider, sil providers, sda accommodation, ndis services perth, ndis providers wa, disability and independent living, independent living solutions, sda apartments, independent living homes, long stay accommodation perth, independent living disabled, a..."
1,0,"adelphi living, adelphi, adelphi living jobs, freedom adelphi, adelphi perth, the adelphi, adelphi hotel perth, adelphi beauty, adelphi insurance, adelphi menu, logo adelphi, 27 adelphi road claremont, air adelphi, adelphi group, adelphi logo, adelphi freedom, adelphi employment, adelphi springs, freedom adelphi table, adelphi management, adelphi ceo, clicks adelphi, adelphi laudale, adelphi s..."
2,1,"sil providers perth, sda providers perth, ndis providers perth, disability service providers perth, ndis service providers perth, it providers perth, disability providers perth, disability employment services providers perth"
3,2,"supported independent living perth, supported independent living ndis, ndis supported independent living, supported independent living, supported independent living providers, supported independent living disability, sil supported independent living, supported independent living provider, supported independent living accommodation, supported independent living wa, supported independent living ..."
4,3,"ndis providers near me, disability services near me, disability support services near me, independent living facilities near me, independent living near me, ndis service providers near me, ndis provider near me, disabled living near me, disability homes near me, disability facility near me, ndis support workers near me, ndis near me, support workers near me, home for the disabled near me, disa..."
5,4,"sil ndis, sil, sil accommodation, sil eligibility, sil perth, sil supports, sil logo, sil ltd, ndia sil, sil living, sil disability, sil assessment ndis, supports sil, sil insurance, sil payments, sil ndia, sil?, /sil, sil., disability sil, www.sil"
6,5,"ndis housing perth, disability housing perth, disability housing, disability rental housing perth, specialist disability housing perth, disability housing options perth, physical disability housing perth, disability housing solutions perth, sil disability housing, independent living disability housing, disability housing wa, disability housing providers, disability housing solutions brisbane, ..."
7,6,"disability supported accommodation perth, supported accommodation perth, supported accommodation ndis, supported accommodation for people with disabilities, supported accommodation, ndis supported accommodation, mental health supported accommodation perth, shared supported accommodation disability services, disability supported accommodation, shared supported accommodation"
8,7,"sil accommodation perth, furnished accommodation perth, share accommodation perth, wheelchair accessible accommodation perth, accessible accommodation perth, mental health accommodation perth, ndis accommodation perth, temporary accommodation perth, accommodation perth region, shared accommodation perth, weekly accommodation perth"
9,8,"ndis, ndis events perth, ndis perth, ndis events, ndis living, ndis mental health perth, able living, ndis properties perth, alchera living, atsa, ndis joondalup, retirement living perth, perth living, ndis rockingham, altum living, ndis bunbury, ndis contact number, ndis number, adventist senior living, ndis air conditioning, ndis tablet, community living perth, howard emery, ndis locations, ..."


## Use the same ML model on each cluster so we can name our clusters automatically

In [6]:
# Create an empty list to store our cluster names
cluster_name = []

# Use Iterrows and tfIdVectorizer to get all words and their TF-IDF Score per cluster
for index, row in clusters_queries.iterrows():
    
    queries = row['Top queries']
    queries = [queries]

    # Add a stop_words function to drop irrelevant words (ex: “a”, “the”, “is”, “are”, "and" etc...)
    my_stop_words = text.ENGLISH_STOP_WORDS

    # Run tfIdfVectorizer for each cluster
    tfIdfVectorizer=TfidfVectorizer(use_idf=True, stop_words=my_stop_words)
    tfIdf = tfIdfVectorizer.fit_transform(queries)
    query_score_cluster = pd.DataFrame(tfIdf[0].T.todense(), index=tfIdfVectorizer.get_feature_names(), columns=["TF-IDF Score"])
    query_score_cluster = query_score_cluster.sort_values('TF-IDF Score', ascending=False)

    # Get the top 3 queries (selected by highest scores) per cluster and get them into lists, clean words of commas and ''
    top_three_queries = query_score_cluster.index[:3]
    top_three_queries_list = top_three_queries.values.tolist()
    top_three_queries_list = str(top_three_queries_list).replace(',', '')
    top_three_queries_list = str(top_three_queries_list).replace('[','').replace(']','')
    top_three_queries_list = str(top_three_queries_list).replace("'","")

    # Append our top 3 queries to our cluster_name list
    cluster_name.append(top_three_queries_list)
    
    # Create a Cluster Name dataframe    
    cluster_name_df = pd.DataFrame({'Cluster Name': cluster_name})

cluster_name_df


,Cluster Name
0,perth ndis living
1,adelphi freedom perth
2,perth providers disability
3,independent living supported
4,near disability ndis
5,sil disability ndia
6,housing disability perth
7,accommodation supported disability
8,accommodation perth accessible
9,living ndis perth


# Merge our dataframes so we can see all of our keywords grouped into different clusters and their names automatically generated. We can make sure it is matching properly

In [7]:
# Merge cluster_name_df with clusters_queries so we can have them matching and double check the cluster names match with our queries
clusters = pd.merge(cluster_name_df, clusters_queries, left_index=True, right_index=True)
clusters

,Cluster Name,Cluster Number,Top queries
0,perth ndis living,-1,"adelphi living mandurah, sil housing, ndis wa providers, adelphi care, assisted living for schizophrenia near me, adelphi housing, ndis support provider, sil providers, sda accommodation, ndis services perth, ndis providers wa, disability and independent living, independent living solutions, sda apartments, independent living homes, long stay accommodation perth, independent living disabled, a..."
1,adelphi freedom perth,0,"adelphi living, adelphi, adelphi living jobs, freedom adelphi, adelphi perth, the adelphi, adelphi hotel perth, adelphi beauty, adelphi insurance, adelphi menu, logo adelphi, 27 adelphi road claremont, air adelphi, adelphi group, adelphi logo, adelphi freedom, adelphi employment, adelphi springs, freedom adelphi table, adelphi management, adelphi ceo, clicks adelphi, adelphi laudale, adelphi s..."
2,perth providers disability,1,"sil providers perth, sda providers perth, ndis providers perth, disability service providers perth, ndis service providers perth, it providers perth, disability providers perth, disability employment services providers perth"
3,independent living supported,2,"supported independent living perth, supported independent living ndis, ndis supported independent living, supported independent living, supported independent living providers, supported independent living disability, sil supported independent living, supported independent living provider, supported independent living accommodation, supported independent living wa, supported independent living ..."
4,near disability ndis,3,"ndis providers near me, disability services near me, disability support services near me, independent living facilities near me, independent living near me, ndis service providers near me, ndis provider near me, disabled living near me, disability homes near me, disability facility near me, ndis support workers near me, ndis near me, support workers near me, home for the disabled near me, disa..."
5,sil disability ndia,4,"sil ndis, sil, sil accommodation, sil eligibility, sil perth, sil supports, sil logo, sil ltd, ndia sil, sil living, sil disability, sil assessment ndis, supports sil, sil insurance, sil payments, sil ndia, sil?, /sil, sil., disability sil, www.sil"
6,housing disability perth,5,"ndis housing perth, disability housing perth, disability housing, disability rental housing perth, specialist disability housing perth, disability housing options perth, physical disability housing perth, disability housing solutions perth, sil disability housing, independent living disability housing, disability housing wa, disability housing providers, disability housing solutions brisbane, ..."
7,accommodation supported disability,6,"disability supported accommodation perth, supported accommodation perth, supported accommodation ndis, supported accommodation for people with disabilities, supported accommodation, ndis supported accommodation, mental health supported accommodation perth, shared supported accommodation disability services, disability supported accommodation, shared supported accommodation"
8,accommodation perth accessible,7,"sil accommodation perth, furnished accommodation perth, share accommodation perth, wheelchair accessible accommodation perth, accessible accommodation perth, mental health accommodation perth, ndis accommodation perth, temporary accommodation perth, accommodation perth region, shared accommodation perth, weekly accommodation perth"
9,living ndis perth,8,"ndis, ndis events perth, ndis perth, ndis events, ndis living, ndis mental health perth, able living, ndis properties perth, alchera living, atsa, ndis joondalup, retirement living perth, perth living, ndis rockingham, altum living, ndis bunbury, ndis contact number, ndis number, adventist senior living, ndis air conditioning, ndis tablet, community living perth, howard emery, ndis locations, ..."


## We only retain our Cluster names and numbers together

In [8]:
# splited_clusters = pd.concat([pd.Series(row['Cluster Name'], row['Top queries'].split(','))              
#                     for _, row in clusters.iterrows()]).reset_index()
cluster_name_nb = clusters[["Cluster Name", "Cluster Number"]]
cluster_name_nb.head()

,Cluster Name,Cluster Number
0,perth ndis living,-1
1,adelphi freedom perth,0
2,perth providers disability,1
3,independent living supported,2
4,near disability ndis,3


In [9]:
# Merge it with our machine learning results
clusters_queries = pd.merge(result, cluster_name_nb, how="inner", on="Cluster Number")
clusters_queries.head() 



,Cluster Number,Top queries,Cluster Name
0,0,adelphi living,adelphi freedom perth
1,0,adelphi,adelphi freedom perth
2,0,adelphi living jobs,adelphi freedom perth
3,0,freedom adelphi,adelphi freedom perth
4,0,adelphi perth,adelphi freedom perth


In [10]:
# Merge it with the original dataset to get the matching query performances
final_df = pd.merge(df, clusters_queries, how="inner", on="Top queries")
final_df

,Top queries,Clicks,Impressions,CTR,Position,Cluster Number,Cluster Name
0,adelphi living,255,765,33.33%,1.06,0,adelphi freedom perth
1,adelphi,15,314,4.78%,10.04,0,adelphi freedom perth
2,sil providers perth,11,98,11.22%,2.20,1,perth providers disability
3,supported independent living perth,8,327,2.45%,8.10,2,independent living supported
4,disability accommodation perth,4,645,0.62%,21.36,22,accommodation disability supported
...,...,...,...,...,...,...,...
790,disabled living manchester,0,1,0%,134.00,-1,perth ndis living
791,disability insurance perth,0,1,0%,141.00,33,disability perth insurance
792,housing for people with disabilities,0,1,0%,196.00,-1,perth ndis living
793,disability support providers,0,1,0%,199.00,20,disability support perth


### We now have a complete dataframe with our queries, performances and corresponding cluster names. 

### It means we can now perform data analysis per cluster, which can be useful especially if you have a lot of queries



## Export our dataframe to a Google Spreadsheet (previously created) so it can be read dynamically with Tableau and Google Data Studio

In [11]:
# Use of gspread to connect our Google Spreadsheet to our final dataframe
scope = ['https://www.googleapis.com/auth/spreadsheets',
        'https://www.googleapis.com/auth/drive']

# Load credentials (not on the github / heroku folder, has to be replaced manually for safety purposes)
creds = Credentials.from_service_account_file("default.json", scopes=scope)
client = gspread.authorize(creds)

# Loading a Google Spreadsheet created before
google_sh = client.open("keyword_clustering")

# Selecting the sheet
sheet1 = google_sh.get_worksheet(0)

# Clearing output
sheet1.clear()

# Send our pandas dataframe to the google spreadsheet (you will be able to load it from Tableau and Google Data Studio)
df_google = set_with_dataframe(sheet1, final_df)

In [12]:
# Read our Google Spreadsheet to confirm the data has been correctly exported ('Unnamed' are the empty cells)
df_google = get_as_dataframe(sheet1)
df_google.head(20)

,Top queries,Clicks,Impressions,CTR,Position,Cluster Number,Cluster Name,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25
0,adelphi living,255.0,765.0,0.3333,1.06,0.0,adelphi freedom perth,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,adelphi,15.0,314.0,0.0478,10.04,0.0,adelphi freedom perth,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,sil providers perth,11.0,98.0,0.1122,2.20,1.0,perth providers disability,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,supported independent living perth,8.0,327.0,0.0245,8.10,2.0,independent living supported,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,disability accommodation perth,4.0,645.0,0.0062,21.36,22.0,accommodation disability supported,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,sda providers perth,4.0,39.0,0.1026,7.21,1.0,perth providers disability,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,ndis providers near me,3.0,54.0,0.0556,1.00,3.0,near disability ndis,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,adelphi living mandurah,3.0,18.0,0.1667,1.06,-1.0,perth ndis living,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,sil ndis,2.0,906.0,0.0022,37.77,4.0,sil disability ndia,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,ndis housing perth,2.0,454.0,0.0044,10.86,5.0,housing disability perth,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Export our final_df Dataframe to sqlite for further purposes (ex: load JSON URL data with Plotly or D3)

In [13]:
# SQL Alchemy
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [14]:
# Create connection
engine = create_engine('sqlite:///keyword_clustering.sqlite', echo=True)
conn = engine.connect()

2021-09-07 10:37:40,785 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2021-09-07 10:37:40,792 INFO sqlalchemy.engine.base.Engine ()
2021-09-07 10:37:40,795 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2021-09-07 10:37:40,797 INFO sqlalchemy.engine.base.Engine ()


In [15]:
# Reflect an existing database into a new model
Base = automap_base()
Base.prepare(engine, reflect=True)

2021-09-07 10:37:40,819 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2021-09-07 10:37:40,822 INFO sqlalchemy.engine.base.Engine ()
2021-09-07 10:37:40,827 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("keyword_clustering")
2021-09-07 10:37:40,830 INFO sqlalchemy.engine.base.Engine ()
2021-09-07 10:37:40,837 INFO sqlalchemy.engine.base.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = 'keyword_clustering' AND type = 'table'
2021-09-07 10:37:40,839 INFO sqlalchemy.engine.base.Engine ()
2021-09-07 10:37:40,844 INFO sqlalchemy.engine.base.Engine PRAGMA main.foreign_key_list("keyword_clustering")
2021-09-07 10:37:40,847 INFO sqlalchemy.engine.base.Engine ()
2021-09-07 10:37:40,850 INFO sqlalchemy.engine.base.Engine PRAGMA temp.foreign_key_list("keyword_clustering")
2021-09-07 10:37:40,852 INFO sqlalchemy.engine.base.Engine ()
2021-09-07 10:37:40,854 INFO sqlalchemy

In [16]:
# Export our dataframe to SQL and replace the data if the table is already there
final_df.to_sql("keyword_clustering", conn, if_exists="replace")

2021-09-07 10:37:40,921 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("keyword_clustering")
2021-09-07 10:37:40,923 INFO sqlalchemy.engine.base.Engine ()
2021-09-07 10:37:40,928 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("keyword_clustering")
2021-09-07 10:37:40,931 INFO sqlalchemy.engine.base.Engine ()
2021-09-07 10:37:40,934 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2021-09-07 10:37:40,936 INFO sqlalchemy.engine.base.Engine ()
2021-09-07 10:37:40,939 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("keyword_clustering")
2021-09-07 10:37:40,941 INFO sqlalchemy.engine.base.Engine ()
2021-09-07 10:37:40,947 INFO sqlalchemy.engine.base.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = 'keyword_clustering' AND type = 'table'
2021-09-07 10:37:40,949 INFO sqlalchemy.engine.base.Engine ()
2021-09-07 10:37:40,952 INFO sqlalchemy.engine.base

In [17]:
# Reflect Database into ORM classes
Base = automap_base()
Base.prepare(engine, reflect=True)
Base.classes.keys()

2021-09-07 10:37:41,098 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2021-09-07 10:37:41,100 INFO sqlalchemy.engine.base.Engine ()
2021-09-07 10:37:41,104 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("keyword_clustering")
2021-09-07 10:37:41,105 INFO sqlalchemy.engine.base.Engine ()
2021-09-07 10:37:41,110 INFO sqlalchemy.engine.base.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = 'keyword_clustering' AND type = 'table'
2021-09-07 10:37:41,113 INFO sqlalchemy.engine.base.Engine ()
2021-09-07 10:37:41,117 INFO sqlalchemy.engine.base.Engine PRAGMA main.foreign_key_list("keyword_clustering")
2021-09-07 10:37:41,119 INFO sqlalchemy.engine.base.Engine ()
2021-09-07 10:37:41,120 INFO sqlalchemy.engine.base.Engine PRAGMA temp.foreign_key_list("keyword_clustering")
2021-09-07 10:37:41,122 INFO sqlalchemy.engine.base.Engine ()
2021-09-07 10:37:41,123 INFO sqlalchemy

[]

In [18]:
# Create our session (link) from Python to the DB
session = Session(engine)

In [19]:
# Use `engine.execute` to select and display the first 10 rows from the station table
engine.execute('SELECT * FROM keyword_clustering LIMIT 5').fetchall()

2021-09-07 10:37:41,179 INFO sqlalchemy.engine.base.Engine SELECT * FROM keyword_clustering LIMIT 5
2021-09-07 10:37:41,180 INFO sqlalchemy.engine.base.Engine ()


[(0, 'adelphi living', 255, 765, '33.33%', 1.06, 0, 'adelphi freedom perth'),
 (1, 'adelphi', 15, 314, '4.78%', 10.04, 0, 'adelphi freedom perth'),
 (2, 'sil providers perth', 11, 98, '11.22%', 2.2, 1, 'perth providers disability'),
 (3, 'supported independent living perth', 8, 327, '2.45%', 8.1, 2, 'independent living supported'),
 (4, 'disability accommodation perth', 4, 645, '0.62%', 21.36, 22, 'accommodation disability supported')]

In [20]:
# Query keyword_clustering in the the Database
data = pd.read_sql("SELECT * FROM keyword_clustering", conn)
data

2021-09-07 10:37:41,196 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("SELECT * FROM keyword_clustering")
2021-09-07 10:37:41,197 INFO sqlalchemy.engine.base.Engine ()
2021-09-07 10:37:41,199 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("SELECT * FROM keyword_clustering")
2021-09-07 10:37:41,201 INFO sqlalchemy.engine.base.Engine ()
2021-09-07 10:37:41,202 INFO sqlalchemy.engine.base.Engine SELECT * FROM keyword_clustering
2021-09-07 10:37:41,203 INFO sqlalchemy.engine.base.Engine ()


,index,Top queries,Clicks,Impressions,CTR,Position,Cluster Number,Cluster Name
0,0,adelphi living,255,765,33.33%,1.06,0,adelphi freedom perth
1,1,adelphi,15,314,4.78%,10.04,0,adelphi freedom perth
2,2,sil providers perth,11,98,11.22%,2.20,1,perth providers disability
3,3,supported independent living perth,8,327,2.45%,8.10,2,independent living supported
4,4,disability accommodation perth,4,645,0.62%,21.36,22,accommodation disability supported
...,...,...,...,...,...,...,...,...
790,790,disabled living manchester,0,1,0%,134.00,-1,perth ndis living
791,791,disability insurance perth,0,1,0%,141.00,33,disability perth insurance
792,792,housing for people with disabilities,0,1,0%,196.00,-1,perth ndis living
793,793,disability support providers,0,1,0%,199.00,20,disability support perth
